# Neo4j - Kit Chin Jie Ying

In [1]:
from data_class.JieYing_neo4jConnection import Neo4jConnection
from pyspark.sql import SparkSession
from uuid import uuid4

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("CSV to Neo4j") \
    .getOrCreate()

# Initialize Neo4j connection
URI = "neo4j+s://2257c2b1.databases.neo4j.io"  ###Remember to replace
neo4j_user = "neo4j"
neo4j_password = "9GMnG4B136sU9Onf8VugQuULe2UMZ7vPjeuD-1nCqbQ" ###Remember to replace

# Create a Neo4jConnection instance
neo4j_conn = Neo4jConnection(uri=URI, user=neo4j_user, password=neo4j_password)

24/12/22 14:25:24 WARN Utils: Your hostname, MSI. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/22 14:25:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/22 14:25:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Read CSV file from HDFS
df = spark.read.csv(
    "lexicon.csv", 
    header=True,         
    inferSchema=True,   
    multiLine=True,     
    sep=",",
    quote='"',          
    escape='"'           
)

# Collect all rows into a list of dictionaries
nodes = df.collect()

# Process each row and create nodes and relationships
for row in nodes:
    kosa_kata = row["Kata"]
    maksud = row["Maksud"]
    terjemahan = row["Terjemahan"]
    golongan_kata = row["Golongan_Kata"]

    # Generate dynamic IDs for each column
    kosa_id = str(uuid4())
    maksud_id = str(uuid4())
    terjemahan_id = str(uuid4())
    golongan_id = str(uuid4())

    # Create Kosa_Kata node
    neo4j_conn.create_kosa_kata(kosa_id, kosa_kata)

    # Create Maksud node
    neo4j_conn.create_maksud(maksud_id, maksud)

    # Create Terjemahan node
    neo4j_conn.create_terjemahan(terjemahan_id, terjemahan)

    # Create Golongan_Kata node
    neo4j_conn.create_golongan_kata(golongan_id, golongan_kata)
    
    # Create relationships
    neo4j_conn.create_relationship(kosa_id, maksud_id, "MEMPUNYAI_MAKNA", "KosaKata", "Maksud", "Id", "MaksudID")
    neo4j_conn.create_relationship(kosa_id, terjemahan_id, "MEMPUNYAI_TERJEMAHAN", "KosaKata", "Terjemahan", "Id", "TerjemahanID")
    neo4j_conn.create_relationship(kosa_id, golongan_id, "TERGOLONG_DALAM", "KosaKata", "GolonganKata", "Id", "GkID")

Created/Matched Kosa_Kata: a
Created/Matched Maksud: (sistem SI) atto-, awalan untuk 10-18., (biologi) nama salah satu golongan darah., (elektrik) simbol untuk ampere, unit arus elektrik., (plat lesen antarabangsa) kod negara Austria., (matematik) simbol nombor heksadesimal untuk 10., (pemakanan) nama salah satu vitamin., (pendidikan) menandakan nilai yang amat baik., (pengkomputeran) alamat lazim untuk pemacu cakera liut., (permainan) menandakan daun sat Ace., (takwim) simbol untuk bulan April atau hari Ahad., Huruf pertama bagi abjad bahasa Melayu, yang ditulis dalam tulisan Rumi., Transkripsi rōmaji bagi あ, Transkripsi rōmaji bagi ア, Kata ganti nama diri tunggal ketiga; dia.
Created/Matched Terjemahan: a
Created/Matched Golongan_Kata: Kata Nama Am
Created/Matched relationship between KosaKata and Maksud
Created/Matched relationship between KosaKata and Terjemahan
Created/Matched relationship between KosaKata and GolonganKata
Created/Matched Kosa_Kata: aa
Created/Matched Maksud: tida

In [3]:
# Close the Neo4j connection and Spark session
neo4j_conn.close()
spark.stop()